1. 네이버 영화 리뷰 데이터셋을 활용하여, 자연어 모델링을 위한 형태소 분석 등 전처리 기법을 학습합니다.

2. 전처리된 자연어 데이터를 벡터화하는 TF-IDF 기법을 이해합니다.

3. 벡터화된 데이터를 활용하여 토픽 모델링과 감성분석 모델링을 실습해봅니다.

In [ ]:
# (구글 코랩에서 실습할 경우) 아래 코드를 실행하여 한글 폰트를 설치합니다. 
# 설치가 완료되면, 커널을 재시작합니다. (상단 메뉴에서 [런타임] - [런타임 다시 시작] 을 선택)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-san

# 라이브러리 불러오기


In [ ]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

# 네이버 영화 리뷰 데이터 - 다운로드
-  출처: https://github.com/e9t/nsmc

In [ ]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
# 데이터 다운로드 
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

'/content/data/review.txt'

In [ ]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv(os.path.join(DATA_DIR, 'review.txt'), sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 한글 형태소 분석

한국어 정보처리를 위한 파이썬 패키지인 KoNLPy(코엔엘파이)를 활용하여

분석 대상이 되는 문장(텍스트)를 최소 의미 단어를 뜻하는 형태소를 기준으로 토큰(token)으로 분리

## **1) KoNLPy 설치**

In [ ]:
# 코랩에 konlpy 패키지를 설치합니다.
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **2) KoNLPy 한글 형태소 분석**

형태소 분석기의 한 종류인 Okt tagger의 다양한 분석 함수를 적용하고, 각 함수의 기능과 차이를 이해합니다.

In [ ]:
# 긍정 리뷰를 하나 선택
sample_text = data['document'].iloc[149978]
print(type(sample_text))
print(sample_text)

<class 'str'>
그리 만족스럽진못했어도 7점은 나와야되는것같아 10점줌. 주인공들연기도 훌륭했고 내용도 이정도면 괜찮았다. 해피엔딩으로끝났으면 그저 진부한영화가 되버릴뻔. 그래도 엔딩이 섭섭한건 없지않아있었음. 킬러들의도시도그렇고 콜린파렐을 너무 축축하게 끝내버리는듯


In [ ]:
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

In [ ]:
# (단어, 품사) 추출
tokens = okt.pos(sample_text)
pprint(tokens)

[('그리', 'Verb'),
 ('만족스럽진못', 'Adjective'),
 ('했어도', 'Verb'),
 ('7', 'Number'),
 ('점', 'Noun'),
 ('은', 'Josa'),
 ('나와야', 'Verb'),
 ('되는것', 'Verb'),
 ('같아', 'Adjective'),
 ('10', 'Number'),
 ('점줌', 'Noun'),
 ('.', 'Punctuation'),
 ('주인공', 'Noun'),
 ('들', 'Suffix'),
 ('연', 'Modifier'),
 ('기도', 'Noun'),
 ('훌륭했고', 'Adjective'),
 ('내용', 'Noun'),
 ('도', 'Josa'),
 ('이', 'Determiner'),
 ('정도', 'Noun'),
 ('면', 'Josa'),
 ('괜찮았다', 'Adjective'),
 ('.', 'Punctuation'),
 ('해피엔딩', 'Noun'),
 ('으로', 'Josa'),
 ('끝났으면', 'Verb'),
 ('그저', 'Adverb'),
 ('진부한', 'Adjective'),
 ('영화', 'Noun'),
 ('가', 'Josa'),
 ('되', 'Verb'),
 ('버릴', 'Verb'),
 ('뻔', 'Noun'),
 ('.', 'Punctuation'),
 ('그래도', 'Adverb'),
 ('엔딩', 'Noun'),
 ('이', 'Josa'),
 ('섭섭한건', 'Adjective'),
 ('없지', 'Adjective'),
 ('않아있었음', 'Verb'),
 ('.', 'Punctuation'),
 ('킬러', 'Noun'),
 ('들', 'Suffix'),
 ('의', 'Josa'),
 ('도시', 'Noun'),
 ('도', 'Josa'),
 ('그렇고', 'Adjective'),
 ('콜린파렐', 'Noun'),
 ('을', 'Josa'),
 ('너무', 'Adverb'),
 ('축축하게', 'Adjective'),
 ('끝내', 'No

In [ ]:
# 단어 ONLY 추출
tokens = okt.morphs(sample_text)
pprint(tokens)

['그리',
 '만족스럽진못',
 '했어도',
 '7',
 '점',
 '은',
 '나와야',
 '되는것',
 '같아',
 '10',
 '점줌',
 '.',
 '주인공',
 '들',
 '연',
 '기도',
 '훌륭했고',
 '내용',
 '도',
 '이',
 '정도',
 '면',
 '괜찮았다',
 '.',
 '해피엔딩',
 '으로',
 '끝났으면',
 '그저',
 '진부한',
 '영화',
 '가',
 '되',
 '버릴',
 '뻔',
 '.',
 '그래도',
 '엔딩',
 '이',
 '섭섭한건',
 '없지',
 '않아있었음',
 '.',
 '킬러',
 '들',
 '의',
 '도시',
 '도',
 '그렇고',
 '콜린파렐',
 '을',
 '너무',
 '축축하게',
 '끝내',
 '버리는듯']


In [ ]:
# 명사 ONLY 추출
tokens = okt.nouns(sample_text)
pprint(tokens)

['점',
 '점줌',
 '주인공',
 '기도',
 '내용',
 '정도',
 '해피엔딩',
 '영화',
 '뻔',
 '엔딩',
 '킬러',
 '도시',
 '콜린파렐',
 '끝내']


## **3) 텍스트 전처리**

Okt 분석 도구 중에서 명사를 추출하는 함수를 사용하여, 각 리뷰 문장에서 명사를 추출하여 토큰화합니다. 이 때, 두 글자 이하의 단어를 분석대상에서 제외하기로 합니다.

In [ ]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 예측력 향상을 위해, 149995개의 전체 샘플 추출
review_data = review_data[:149995]

# 배열의 크기
print(review_data.shape)

# 첫번째 데이터
print(review_data[0])

# 마지막 데이터
print(review_data[149994])

(149995,)
아 더빙.. 진짜 짜증나네요 목소리
한국 영화 최초로 수간하는 내용이 담긴 영화


In [ ]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 149995/149995 [09:54<00:00, 252.14it/s]

149995
목소리


## 4**) TF-IDF 벡터로 표현**

LDA, 로지스틱 회귀 등 머신러닝의 입력값은 수치형태로 표현된 매트릭스입니다. 머신러닝 모델을 활용하기 위해 이 프로젝트에서는 TF-IDF의 형태로 텍스트 문서를 수치화 합니다.


In [ ]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(149995, 17190)
  (0, 4849)	1.0


In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

17190
{'목소리': 4849, '포스터': 15807, '교도소': 976, '이야기': 11680, '스파이더맨': 8274}


In [ ]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{4849: '목소리', 15807: '포스터', 976: '교도소', 11680: '이야기', 8274: '스파이더맨'}


In [ ]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'목소리'

# **Topic Modeling**

문서는 여러 종류의 단어들의 집합체로 구성됩니다. 문서를 구성하는 단어들의 확률분포를 활용하여 문서의 토픽(주제)를 파악할 수 있습니다


## **1) LDA (LatentDirichletAllocation)**

LDA 토픽 모델링을 이용하여, 문서로부터 두 종류의 토픽을 추출합니다. 문서의 감성(긍정, 부정) 형태로 토픽이 추출되는지, 아니면 다른 종류의 토픽이 추출되는지 분석하세요.


In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽)
print(lda.components_.shape)

(2, 17190)


In [ ]:
# 21개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('스토리', 3082.636), ('드라마', 2777.913), ('마지막', 1686.877), ('이야기', 1387.648), ('주인공', 1344.327), ('코미디', 741.739), ('연기력', 618.403), ('스릴러', 564.521), ('공포영화', 491.146), ('제대로', 490.669)]
토픽 유형 2: [('쓰레기', 2271.987), ('이영화', 1168.633), ('시리즈', 608.106), ('캐릭터', 606.595), ('긴장감', 563.977), ('지루함', 475.648), ('한국영', 465.373), ('영화관', 434.144), ('우리나라', 411.584), ('분위기', 406.407)]


## **2) pyLDAvis 시각화**

pyLDAvis 패키지를 활용하여, LDA 토픽 모델링 경과를 시각화 합니다.

In [ ]:
# pyLDAvis 설치
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

## **3) 추가 LDA** (토픽 3개)




In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 3으로 설정)
lda2 = LatentDirichletAllocation(n_components=3)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda2.fit(review_tfid)

LatentDirichletAllocation(n_components=3)

In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (3개의 토픽)
print(lda2.components_.shape)

(3, 17190)


In [ ]:
# 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda2.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('드라마', 2778.469), ('이영화', 1168.65), ('공포영화', 491.871), ('킬링타임', 433.706), ('우리나라', 411.522), ('포스터', 402.967), ('몰입도', 361.821), ('캐스팅', 358.416), ('줄거리', 246.465), ('목소리', 211.486)]
토픽 유형 2: [('이야기', 1388.336), ('주인공', 1344.326), ('코미디', 741.633), ('캐릭터', 615.842), ('시리즈', 607.997), ('스릴러', 564.461), ('긴장감', 563.923), ('지루함', 475.526), ('영화로', 467.37), ('한국영', 465.243)]
토픽 유형 3: [('스토리', 3082.476), ('쓰레기', 2272.806), ('마지막', 1690.039), ('연기력', 623.037), ('제대로', 490.673), ('도대체', 480.631), ('최고다', 446.526), ('애니메이션', 387.304), ('갈수록', 354.442), ('비디오', 321.021)]


In [ ]:
# LDA2 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda2, review_tfid, tfid)
pyLDAvis.display(visualization)

# **Sentimental Analysis (감성 분석)**

감성 분석은 문장(텍스트)에 드러나는 사람들의 생각이나 느낌, 성향과 같이 주관적인 감성을 분석하는 자연어 처리 기법입니다. 머신러닝 분류 모델을 활용하여 문장이 갖는 긍정 또는 부정의 감성을 예측하세요.

## 1) 머신러닝 알고리즘을 활용하여, 긍정/부정 감성을 분류

In [ ]:
labels = data['label'].iloc[:149995].values
print(labels.shape)
print(labels[:5])

(149995,)
[0 1 0 0 1]


In [ ]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [ ]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


## 2) Part 1에서 수집한 리뷰를 긍정, 부정으로 감성 분류 예측 - test


In [ ]:
# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('/content/drive/MyDrive/HUFS Data Analysis/naver_review_Avatar.csv')

test.head()

,Unnamed: 0,title,user,review
0,0,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...",insu****,"\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티..."
1,1,2014년에 이글을 읽는사람들에게..,xnzm****,"\n2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n..."
2,2,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",alwa****,\n영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대...
3,3,"아바타2 3,4 제작소식 모음",soqu****,\n \n \n \n<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n...
4,4,아바타2 스토리,doub****,\n전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리...


In [ ]:
# 첫 번째 리뷰를 선택
test_sample = test['review'][0]
test_sample

'\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴\n개봉\n2017 미국\n\n리뷰보기\n\n\n\u200b*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연: 조 셀다나 (네이티리 역), 샘 워싱턴 (제이크 설리 역), 시고니 위버 (그레이스 어거스틴 역), 스티븐 랭 (마일즈 쿼리치 역)...국적: 미국북미 개봉일: 2017년 12월 25일2017년 12월 25일 북미 개봉작으로 기대를 모으고 있는 영화 아바타 2...특히나 주인공 "네이티리" 의 캐릭터 이미지가 공개가 되어서 올려봅니다.배우 조 샐다나와 샘 워싱턴, 시고니 위버 그리고 스티븐 랭까지 전편에서 출연했었던 배우들이 그대로 출연을 하고 있어서더욱 기대가 되는 영화인 것 같습니다. 아무튼 제임스 카메론 감독의 영화 아바타 2편을 시작으로2018년 아바타 3와 2019년 아바타 4까지 진행되고 있어서 앞으로 더욱 기대되는 영화 아바타인 것 같습니다.\n'

In [ ]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

'아바타감독제임스카메론출연시고니위버스티븐랭조샐다나샘워싱턴개봉미국리뷰보기아바타장르판타지모험액션감독제임스카메론주연조셀다나네이티리역샘워싱턴제이크설리역시고니위버그레이스어거스틴역스티븐랭마일즈쿼리치역국적미국북미개봉일년월일년월일북미개봉작으로기대를모으고있는영화아바타특히나주인공네이티리의캐릭터이미지가공개가되어서올려봅니다배우조샐다나와샘워싱턴시고니위버그리고스티븐랭까지전편에서출연했었던배우들이그대로출연을하고있어서더욱기대가되는영화인것같습니다아무튼제임스카메론감독의영화아바타편을시작으로년아바타와년아바타까지진행되고있어서앞으로더욱기대되는영화아바타인것같습니다'

In [ ]:
# 세 글자 이상의 명사만을 추출

tokens = okt.nouns(test_sample)
cleaned_tokens = []
for word in tokens:
    if len(word) > 2:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_test_review = " ".join(cleaned_tokens)

cleaned_test_review

'아바타 제임스 카메론 워싱턴 아바타 판타지 제임스 카메론 네이티 리역샘 워싱턴 제이크 레이스 어거스틴 스티븐 개봉일 아바타 네이티 캐릭터 이미지 워싱턴 스티븐 영화인 제임스 카메론 아바타 아바타 아바타 아바타'

In [ ]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_test_review])

# 배열을 출력
print(test_review_tfid)

  (0, 15568)	0.0808074257302683
  (0, 14642)	0.0684315678085225
  (0, 14553)	0.31364617325609667
  (0, 13166)	0.2919564730016513
  (0, 13165)	0.12011378206990456
  (0, 11515)	0.09148665090248491
  (0, 10958)	0.3197320119039558
  (0, 10285)	0.07656065146130983
  (0, 8968)	0.7282077153097543
  (0, 8253)	0.19797040447251155
  (0, 3880)	0.1173603983808489
  (0, 2403)	0.279154297914368
  (0, 322)	0.11449110353079545


In [ ]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))

분석 결과 긍정적인 리뷰로 예측됩니다. 


## 3) Part1에서 수집한 전체 리뷰 - 감성 분류 예측

In [ ]:
# 판다스 데이터프레임로 읽어오기
df = pd.read_csv('/content/drive/MyDrive/HUFS Data Analysis/naver_review_Avatar.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  71 non-null     int64 
 1   title       71 non-null     object
 2   user        71 non-null     object
 3   review      71 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.3+ KB


,Unnamed: 0,title,user,review
0,0,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...",insu****,"\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티..."
1,1,2014년에 이글을 읽는사람들에게..,xnzm****,"\n2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n..."
2,2,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",alwa****,\n영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대...
3,3,"아바타2 3,4 제작소식 모음",soqu****,\n \n \n \n<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n...
4,4,아바타2 스토리,doub****,\n전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리...


In [ ]:
cleaned_tokens = []

for review in tqdm(df["review"]):

    # 한글을 제외하고 전부 제거
    text = re.sub(r"[^가-힣]", "", review)

    # 세글자 이상의 명사만을 추출
    tokens = okt.nouns(text)

    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)

    cleaned_review

100%|██████████| 71/71 [39:24<00:00, 33.30s/it]


In [ ]:
# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(review_tfid)

  (0, 17165)	0.005110557507557496
  (0, 17164)	0.0025892651637636076
  (0, 17157)	0.0022707059680844824
  (0, 17137)	0.0019547778258255676
  (0, 17133)	0.002503388839186343
  (0, 17128)	0.0025892651637636076
  (0, 17127)	0.0025892651637636076
  (0, 17122)	0.00308679503390338
  (0, 17065)	0.00209125075821024
  (0, 17002)	0.002356582292661747
  (0, 16984)	0.014208643104428403
  (0, 16978)	0.005006777678372686
  (0, 16929)	0.0019394605211222548
  (0, 16865)	0.00552717694040231
  (0, 16808)	0.007069746877985242
  (0, 16793)	0.005178530327527215
  (0, 16786)	0.018342207872843602
  (0, 16771)	0.0025892651637636076
  (0, 16761)	0.0018724611997744734
  (0, 16757)	0.004162795909786946
  (0, 16748)	0.0036786006608551334
  (0, 16707)	0.0022097757461371514
  (0, 16697)	0.0024424586172390114
  (0, 16644)	0.0022282045014180686
  (0, 16631)	0.0020460163881208643
  :	:
  (0, 628)	0.0025892651637636076
  (0, 571)	0.002015708047772899
  (0, 564)	0.0034235226811675677
  (0, 549)	0.003806645070829536
  (0

In [ ]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
review_pred = lr.predict(review_tfid)
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if review_pred > 0 else "부정"))

분석 결과 긍정적인 리뷰로 예측됩니다. 
